# Dataset View Demo
Demonstrates reading a dataset table (from Avro) w/ a variety of sorts and filters

In [8]:
import os
import pathlib

import pyspark.sql
import pyspark.sql.functions

import sys; sys.path.insert(0, ".")
from sample_data import DATA_DIR, fetch_dataset

In [7]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.12:3.3.2 pyspark-shell'

spark = pyspark.sql.SparkSession.builder.appName(
    "dataset_view_demo"
).getOrCreate()

In [3]:
# download dataset to use
DATASET_TO_USE = "Tweets-100M.csv"
DATASET_PATH = fetch_dataset(DATASET_TO_USE)

In [5]:
# load dataset to dataframe
dataset_df = spark.read.option("header", True).format("csv").load(str(DATASET_PATH))
dataset_df.show()

# save dataset to Avro
AVRO_DATASET_PATH = DATASET_PATH.with_suffix(".avro")
dataset_df.write.format("avro").save(str(AVRO_DATASET_PATH), mode="overwrite")

+--------------------+-----------------+----------------------------+--------------+-------------------------+--------------+----------------------+---------------+-------------------+-------------+--------------------+-----------+--------------------+--------------------+--------------------+
|            tweet_id|airline_sentiment|airline_sentiment_confidence|negativereason|negativereason_confidence|       airline|airline_sentiment_gold|           name|negativereason_gold|retweet_count|                text|tweet_coord|       tweet_created|      tweet_location|       user_timezone|
+--------------------+-----------------+----------------------------+--------------+-------------------------+--------------+----------------------+---------------+-------------------+-------------+--------------------+-----------+--------------------+--------------------+--------------------+
|                   0|          neutral|                         1.0|          null|                     null|Virgi

## Sort/Select Demo
The following demo shows how to run select queries on several columns of the dataset. This mimics the operations that *will* be used in the dataset view page of Cleanlab Studio.

#### Notes
Spark does not support offsets, so we have to add a monotonically increasing ID on our sorted table and do a filter to achieve a limit + offset operation.

### Single Column Sort + Select

In [17]:
# load spark table from avro
dataset_table = spark.read.format("avro").load(str(AVRO_DATASET_PATH))

# sort on a single column, and select N samples from an offset
col_name = "tweet_created"
num_samples = 100
offset = 8000

sorted_table = dataset_table.sort(col_name)
sorted_table = sorted_table.withColumn("__sort_id", pyspark.sql.functions.monotonically_increasing_id())
res = sorted_table.where(pyspark.sql.functions.col("__sort_id").between(offset, offset + num_samples))
res.show()

+--------------------+--------------------+----------------------------+--------------------+-------------------------+--------------------+----------------------+-------------+-------------------+-------------+--------------------+--------------------+-------------+--------------------+------------------+---------+
|            tweet_id|   airline_sentiment|airline_sentiment_confidence|      negativereason|negativereason_confidence|             airline|airline_sentiment_gold|         name|negativereason_gold|retweet_count|                text|         tweet_coord|tweet_created|      tweet_location|     user_timezone|__sort_id|
+--------------------+--------------------+----------------------------+--------------------+-------------------------+--------------------+----------------------+-------------+-------------------+-------------+--------------------+--------------------+-------------+--------------------+------------------+---------+
|Please see a JetB...|                null|   

### Multi-column Sort + Select

In [18]:
# load spark table from avro
dataset_table = spark.read.format("avro").load(str(AVRO_DATASET_PATH))

# sort on multiple columns, and select N samples from an offset
col_names = ["tweet_created", "airline_sentiment", "tweet_id"]
num_samples = 100
offset = 8000

sorted_table = dataset_table.sort(col_names)
sorted_table = sorted_table.withColumn("__sort_id", pyspark.sql.functions.monotonically_increasing_id())
res = sorted_table.where(pyspark.sql.functions.col("__sort_id").between(offset, offset + num_samples))
res.show()

+--------+-----------------+----------------------------+--------------------+-------------------------+---------+----------------------+---------------+-------------------+-------------+--------------------+--------------------+-------------+--------------------+--------------------+---------+
|tweet_id|airline_sentiment|airline_sentiment_confidence|      negativereason|negativereason_confidence|  airline|airline_sentiment_gold|           name|negativereason_gold|retweet_count|                text|         tweet_coord|tweet_created|      tweet_location|       user_timezone|__sort_id|
+--------+-----------------+----------------------------+--------------------+-------------------------+---------+----------------------+---------------+-------------------+-------------+--------------------+--------------------+-------------+--------------------+--------------------+---------+
|  324800|         negative|                         1.0|Customer Service ...|                      1.0|   Unite

### Single Column Filter + Sort + Select

In [ ]:
# load spark table from avro
dataset_table = spark.read.format("avro").load(str(AVRO_DATASET_PATH))

# filter on a single column, sort on a single column, and select N samples from an offset
filter_col_name = "airline_sentiment"
filter_col_value = "neutral"
sort_col_name = "tweet_created"
num_samples = 100
offset = 8000

filtered_table = dataset_table.filter(pyspark.sql.functions.col(filter_col_name).eqNullSafe(filter_col_value))
sorted_table = filtered_table.sort(col_name)
sorted_table = sorted_table.withColumn("__sort_id", pyspark.sql.functions.monotonically_increasing_id())
res = sorted_table.where(pyspark.sql.functions.col("__sort_id").between(offset, offset + num_samples))
res.show()

+--------+-----------------+----------------------------+--------------+-------------------------+-------+----------------------+----------+-------------------+-------------+-----------------+-----------+--------------------+--------------+-------------+---------+
|tweet_id|airline_sentiment|airline_sentiment_confidence|negativereason|negativereason_confidence|airline|airline_sentiment_gold|      name|negativereason_gold|retweet_count|             text|tweet_coord|       tweet_created|tweet_location|user_timezone|__sort_id|
+--------+-----------------+----------------------------+--------------+-------------------------+-------+----------------------+----------+-------------------+-------------+-----------------+-----------+--------------------+--------------+-------------+---------+
|    4117|          neutral|                         1.0|          null|                     null| United|                  null|boombaby55|               null|            0|@united no thanks|       null|2

### Mutli Column Filter + Sort + Select

In [23]:
# load spark table from avro
dataset_table = spark.read.format("avro").load(str(AVRO_DATASET_PATH))

# filter on a multiple columns, sort on multiple columns, and select N samples from an offset
filter_col_names = ["airline_sentiment", "user_timezone"]
filter_col_values = ["positive", None]
sort_col_names = ["tweet_created", "airline_sentiment", "tweet_id"]
num_samples = 100
offset = 8000

filtered_table = dataset_table.filter(
    pyspark.sql.functions.col(filter_col_names[0]).eqNullSafe(filter_col_values[0])
).filter(
    pyspark.sql.functions.col(filter_col_names[1]).eqNullSafe(filter_col_values[1]),
)
sorted_table = filtered_table.sort(sort_col_names)
sorted_table = sorted_table.withColumn("__sort_id", pyspark.sql.functions.monotonically_increasing_id())
res = sorted_table.where(pyspark.sql.functions.col("__sort_id").between(offset, offset + num_samples))
res.show()

+--------+-----------------+----------------------------+--------------+-------------------------+----------+----------------------+------------+-------------------+-------------+--------------------+-----------+--------------------+--------------+-------------+---------+
|tweet_id|airline_sentiment|airline_sentiment_confidence|negativereason|negativereason_confidence|   airline|airline_sentiment_gold|        name|negativereason_gold|retweet_count|                text|tweet_coord|       tweet_created|tweet_location|user_timezone|__sort_id|
+--------+-----------------+----------------------------+--------------+-------------------------+----------+----------------------+------------+-------------------+-------------+--------------------+-----------+--------------------+--------------+-------------+---------+
|  333047|         positive|                         1.0|          null|                     null|US Airways|                  null|pamela_moats|               null|            0|@U